In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [2]:
# Transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the training and test datasets
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 47819723.87it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 22382796.35it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden




100%|██████████| 1648877/1648877 [00:00<00:00, 22280075.50it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3476373.86it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)  # Input layer to hidden layer
        self.fc2 = nn.Linear(128, 10)     # Hidden layer to output layer

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the image
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = SimpleNN()


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)


In [7]:
num_epochs = 5

for epoch in range(num_epochs):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')


[Epoch 1, Batch 100] loss: 0.943
[Epoch 1, Batch 200] loss: 0.444
[Epoch 1, Batch 300] loss: 0.350
[Epoch 1, Batch 400] loss: 0.335
[Epoch 1, Batch 500] loss: 0.315
[Epoch 1, Batch 600] loss: 0.274
[Epoch 1, Batch 700] loss: 0.267
[Epoch 1, Batch 800] loss: 0.252
[Epoch 1, Batch 900] loss: 0.215
[Epoch 2, Batch 100] loss: 0.197
[Epoch 2, Batch 200] loss: 0.185
[Epoch 2, Batch 300] loss: 0.192
[Epoch 2, Batch 400] loss: 0.174
[Epoch 2, Batch 500] loss: 0.181
[Epoch 2, Batch 600] loss: 0.154
[Epoch 2, Batch 700] loss: 0.179
[Epoch 2, Batch 800] loss: 0.174
[Epoch 2, Batch 900] loss: 0.153
[Epoch 3, Batch 100] loss: 0.147
[Epoch 3, Batch 200] loss: 0.128
[Epoch 3, Batch 300] loss: 0.126
[Epoch 3, Batch 400] loss: 0.138
[Epoch 3, Batch 500] loss: 0.132
[Epoch 3, Batch 600] loss: 0.136
[Epoch 3, Batch 700] loss: 0.108
[Epoch 3, Batch 800] loss: 0.122
[Epoch 3, Batch 900] loss: 0.111
[Epoch 4, Batch 100] loss: 0.104
[Epoch 4, Batch 200] loss: 0.100
[Epoch 4, Batch 300] loss: 0.117
[Epoch 4, 

In [8]:
correct = 0
total = 0

# Since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # Calculate outputs by running images through the network
        outputs = net(images)
        # The class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


Accuracy of the network on the 10000 test images: 96.77%


In [9]:
# Save the model
PATH = './mnist_net.pth'
torch.save(net.state_dict(), PATH)

# To load the model later
net = SimpleNN()
net.load_state_dict(torch.load(PATH))


<All keys matched successfully>